# Lab 1: Langfuse Tracing

In this lab, we will learn how to use Langfuse Tracing to log and analyze the execution of your LLM applications. The Langfuse is self-hosted on AWS but there is a cloud version available.

[Tracing](https://langfuse.com/docs/tracing) in Langfuse is a way to log and analyze the execution of your LLM applications. The following reference provides a detailed overview of the data model used. It is inspired by OpenTelemetry.


## Traces and Observations:
A trace typically represents a single request or operation. It contains the overall input and output of the function, as well as metadata about the request, such as the user, the session, and tags. Usually, a trace corresponds to a single api call of an application.

Each trace can contain multiple observations to log the individual steps of the execution.

- Observations are of different types:
    - Events are the basic building blocks. They are used to track discrete events in a trace.
    - Spans represent durations of units of work in a trace.
    - Generations are spans used to log generations of AI models. They contain additional attributes about the model, the prompt, and the completion. For generations, token usage and costs are automatically calculated.
- Observations can be nested.

![Trace and Observations](./images/trace-observation.png)
[Source](https://langfuse.com/docs/tracing)

<!-- Adding an screenshot of the trace and observations -->

## Sessions
Optionally, traces can be grouped into sessions. Sessions are used to group traces that are part of the same user interaction. A common example is a thread in a chat interface.
Please refer to the [Sessions documentation](https://langfuse.com/docs/sessions) to add sessions to your traces.
![Trace and Sessions](./images/trace-sessions.png)
[Source](https://langfuse.com/docs/tracing)
<!-- Adding an screenshot of the trace and sessions -->


## Scores[PENDING]: This better to be introduced in the context of evaluationd section


### Lab Setting

Please make sure you have completed the prerequisites to setup the Langfuse project and API keys otherwise xxxx;
Also check custom model pricing otherwise please add the custom model pricing to the Langfuse project. Link to the section;

### Set Langfuse Credentials

In [10]:
import os
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." # Your Langfuse project secret key
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." # Your Langfuse project public key
os.environ["LANGFUSE_HOST"] = "https://xx.cloud.langfuse.com" # Region-specific Langfuse domain

#### Install Python package & import all the necessary packages

We will use the langfuse, boto3:
- The langfuse Python SDK along with the self-hosting deployment to debug and improve LLM applications by tracing model invocations, managing prompts / models configurations and running evaluations.
- The boto3 SDK to interact with models on Amazon Bedrock or Amazon SageMaker.
- Langfuse python SDK 


Run the following command to install the required Python SDKs:

In [ ]:
%pip install langfuse==2.54.1 boto3==1.35.70 litellm==1.52.16

In [7]:
# import all the necessary packages
import boto3
from typing import List, Dict, Optional, Any
from langfuse.decorators import observe, langfuse_context
from botocore.exceptions import ClientError


In [ ]:

# used to access Bedrock configuration
# region has to be in us-west-2 for this lab
bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-west-2"
)
 
# used to invoke the Bedrock Converse API
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-west-2"
)

# Check which models are available in your account
models = bedrock.list_inference_profiles()
for model in models["inferenceProfileSummaries"]:
  print(model["inferenceProfileName"] + " - " + model["inferenceProfileId"])
#  Coverage, log level, etc.

##### Bedrock Converse API
You can use the Amazon Bedrock Converse API to create conversational applications that send and receive messages to and from an Amazon Bedrock model. For example, you can create a chat bot that maintains a conversation over many turns and uses a persona or tone customization that is unique to your needs, such as a helpful technical support assistant.

To use the Converse API, you use the Converse or ConverseStream (for streaming responses) operations to send messages to a model. It is possible to use the existing base inference operations (InvokeModel or InvokeModelWithResponseStream) for conversation applications. However, we recommend using the Converse API as it provides consistent API, that works with all Amazon Bedrock models that support messages. This means you can write code once and use it with different models. Should a model have unique inference parameters, the Converse API also allows you to pass those unique parameters in a model specific structure.

For more details, please refer to the [Carry out a conversation with the Converse API operations](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).

#### Build a wrapper for Bedrock SDK with Converse API

In [9]:
@observe(as_type="generation", name="Bedrock Converse")
def bedrock_converse_langfuse_wrapper(**kwargs):
    # 1. Extract the conversation history from the kwargs
    kwargs_clone = kwargs.copy()
    message = kwargs_clone.pop("message", None)
    modelId = kwargs_clone.pop("modelId", None)
    model_parameters = {
        **kwargs_clone.pop("inferenceConfig", {}),
        **kwargs_clone.pop("additionalModelRequestFields", {}),
    }
    langfuse_context.update_current_observation(
        input=message,
        model=modelId,
        model_parameters=model_parameters,
        metadata= kwargs_clone
    )
    
    # 2. Invoke the Bedrock Converse API
    try:
        response = bedrock_runtime.converse(**kwargs)
    except (ClientError, Exception) as e:
        error_message = f"Error invoking Bedrock Converse API: Can't invoke 'modelId' with reason {e}"
        langfuse_context.update_current_observation( level="ERROR", status_message=error_message)
        return {
            "error": error_message,
            "type": type(e).__name__,
            "statusCode": 500
        }
    # 3. handle the response
    response_text = response["output"]["message"]["content"][0]["text"]
    langfuse_context.update_current_observation(
        output=response_text,
        usage = {
            "input": response["usage"]["inputTokens"],
            "output": response["usage"]["outputTokens"],
            "total": response["usage"]["totalTokens"]
        },
        metadata= {
            "ResponseMetadata": response["ResponseMetadata"],
        }
    )
    return {
        "response": response_text,
        "statusCode": 200
    }

#### Run a test case

In [8]:

modelId = "us.amazon.nova-pro-v1:0"

# Converesation according to AWS spec including prompting + history
user_message = """You will be acting as an AI personal finance advisor named Alex, created by the company SmartFinance Advisors. Your goal is to provide financial advice and guidance to users. You will be replying to users who are on the SmartFinance Advisors site and who will be confused if you don't respond in the character of Alex.
 
Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
User: Hi Alex, I'm really looking forward to your advice!
Alex: Hello! I'm Alex, your AI personal finance advisor from SmartFinance Advisors. How can I assist you with your financial goals today?
</history>
 
Here are some important rules for the interaction:
-  Always stay in character, as Alex, an AI from SmartFinance Advisors.
-  If you are unsure how to respond, say "I'm sorry, I didn't quite catch that. Could you please rephrase your question?"
"""

#system prompt??

#user prompt
messages = [
    {"role": "user", 
    "content": [{"text": user_message}]}
]

response = bedrock_converse_langfuse_wrapper(
    modelId=modelId,
    message=messages
)

print(response)


NameError: name 'bedrock_converse_langfuse_wrapper' is not defined

In [ ]:
#TODO
# 1. Add a system prompt
# adding tags
# adding guardrails
# explain metadata
# adding users 
# adding sessions???
# adding prompt management

